In [ ]:
import pandas as pd

import glob

In [ ]:
voter_history_directory = r'D:\Users\bengen343\Documents\Master Voting History'

In [ ]:
# Load vote history data frame
print("Beginning to assemble vote history file.")

allFiles = glob.glob(voter_history_directory + "/*.txt")
history_df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, sep=",", encoding='cp437', index_col=None, header=0, low_memory=False)
    list_.append(df)
history_df = pd.concat(list_)

history_df.reset_index(drop=True, inplace=True)